# Dory's Story

## Import Packages

In [5]:
import pandas as pd
import zipfile
import requests
import io
import arcpy

## Set Working Folder

In [7]:
dory_working = r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory"

## Call in XY Coordinates

In [5]:
startXY = pd.read_csv(r"C:\Users\tande\Documents\GitHub\GIS5571\Lab 02\Dory Farm XY.csv")
endXY = pd.read_csv(r"C:\Users\tande\Documents\GitHub\GIS5571\Lab 02\North Picnic XY.csv")

## Import Data 

In [20]:
# County boundaries
county_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/bdry_counties/shp_bdry_counties.zip'
county_post_request = requests.post(county_url)
county_post_request

county_zipped = zipfile.ZipFile(
        io.BytesIO(
            county_post_request.content)
)

county_zipped.extractall(dory_working)
county = 'County_Boundaries_in_Minnesota.shp'

# MN Landcover
landcover_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip'
landcover_post_request = requests.post(landcover_url)
landcover_post_request

landcover_zipped = zipfile.ZipFile(
        io.BytesIO(
            landcover_post_request.content)
)

landcover_zipped.extractall(dory_working)
landcover = "NLCD 2019 - Land Cover.lyrx"

# Water routes

water_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/water_strahler_stream_order/shp_water_strahler_stream_order.zip'
water_post_request = requests.post(water_url)
water_post_request

water_zipped = zipfile.ZipFile(
        io.BytesIO(
            water_post_request.content)
)

water_zipped.extractall(dory_working)

stream = 'streams_with_strahler_stream_order.shp'

# Elevation 

elevation_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip'
elevation_post_request = requests.post(elevation_url)
elevation_post_request

elevation_zipped = zipfile.ZipFile(
        io.BytesIO(
            elevation_post_request.content)
)

elevation_zipped.extractall(dory_working)

elevation = 'Digital Elevation Model - 30m Resolution.lyr'

roads_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/trans_roads_mndot_tis/shp_trans_roads_mndot_tis.zip'
roads_post_request = requests.post(roads_url)
roads_post_request

roads_zipped = zipfile.ZipFile(
        io.BytesIO(
            roads_post_request.content)
)

roads_zipped.extractall(dory_working)

roads = 'STREETS_LOAD.shp'

In [2]:
# XY Coordinates to Points

arcpy.management.XYTableToPoint("Dory Farm XY.csv", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\DoryFarmXY_XYTableToPoint1", "Y", "X", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')
arcpy.management.XYTableToPoint("North Picnic XY.csv", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\NorthPicnicXY_XYTableToPoint1", "Y", "X", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

<Result 'C:\\Users\\tande\\Documents\\GIS5571\\GIS5571 Lab 2\\Dory\\GIS5571_Lab2Part2\\GIS5571_Lab2Part2.gdb\\NorthPicnicXY_XYTableToPoint1'>

## Clipping to Extent

In [27]:
# The extent I am using are the 3 counties surrounding the points: Olmstead, Winona, and Wabasha

# Add all data to ArcPro

# Selecting this extent
arcpy.management.SelectLayerByAttribute(county, "NEW_SELECTION", "COUNTY_NAM = 'Wabasha' And COUNTY_NAM = 'Olmsted' And COUNTY_NAM = 'Winona'", None)

# Clipping the boundaries to the selected boundaries
arcpy.analysis.PairwiseClip("County_Boundaries_in_Minnesota", "County_Boundaries_in_Minnesota", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Selected_County_Boundaries", None)

# Clipping streams to selected boundaries
arcpy.analysis.PairwiseClip('streams_with_strahler_stream_order', "County_Boundari_PairwiseClip", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\streams_with_st_PairwiseClip", None)

# Clipping DEM
arcpy.management.Clip(r"Digital Elevation Model - 30m Resolution\Digital Elevation Model", "524956.361 4853464.52 637943.77 4922616.7", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\DigitalElevationModel_Clip", "County_Boundari_PairwiseClip", "32767", "NONE", "MAINTAIN_EXTENT")

# Clipping Land Cover
arcpy.management.Clip("NLCD 2019 - Land Cover", "524956.361 4853464.52 637943.77 4922616.7", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\NLCD2019LandCover_Clip", "County_Boundari_PairwiseClip", "255", "NONE", "NO_MAINTAIN_EXTENT")

# Clipping roads
arcpy.analysis.PairwiseClip("STREETS_LOAD", "County_Boundari_PairwiseClip", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\STREETS_LOAD_PairwiseClip1", None)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000840: The value is not a Mosaic Dataset.
ERROR 000840: The value is not a Mosaic Layer.
ERROR 000840: The value is not a Raster Dataset.
ERROR 000840: The value is not a Raster Layer.
WARNING 000725: Output Raster Dataset: Dataset C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\digital_elevation_model_Clip already exists.
Failed to execute (Clip).


## Preparing Each Clipped Dataset for Raster Calculator

In [ ]:
# Reclassifying Ref https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/creating-a-cost-surface-raster.htm

# For commonalities in rasters - using 9 classifications and Red-Yellow-Green continuous symbology 

# I don't think symbology matters but just doing this for good measure

### County Boundaries

In [ ]:
# Only used for clipping the extent

### Road Network

In [28]:
# Create a 100m buffer around all roads in the extent

arcpy.analysis.PairwiseBuffer("STREETS_LOAD_PairwiseClip1", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\STREETS_LOAD_P_PairwiseBuffe", "100 Meters", "NONE", None, "PLANAR", "0 Meters")

# Ready to be used to clip rivers (to remove any rivers flowing under bridges)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Features: Dataset STREETS_LOAD does not exist or is not supported
WARNING 000725: Output Feature Class: Dataset C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\STREETS_LOAD_PairwiseClip1 already exists.
Failed to execute (PairwiseClip).


### Streams

In [ ]:
# Create a 100m buffer around all streams in the extent

arcpy.analysis.PairwiseBuffer("streams_with_st_PairwiseClip", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\streams_with_s_PairwiseBuffe", "100 Meters", "NONE", None, "PLANAR", "0 Meters") 

# Finding where rivers and roads intersect

arcpy.analysis.PairwiseIntersect("streams_with_s_PairwiseBuffe;STREETS_LOAD_P_PairwiseBuffe", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\streams_street_intersect", "ALL", None, "INPUT")

# Erasing rivers where roads overlap

arcpy.analysis.PairwiseErase("streams_with_s_PairwiseBuffe", "streams_street_intersect", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\streams_street_PairwiseErase", None)

# Merge the streams dataset with the boundaries to identify any no value cells

arcpy.management.Merge("County_Boundari_PairwiseClip;streams_street_PairwiseErase", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\streams_street_Pairwis_Merge", None, "NO_SOURCE_INFO")

# Converting to raster

arcpy.conversion.FeatureToRaster("streams_street_Pairwis_Merge", "SO_VALUE", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\stream_raster", 277.382114)

# Reclassify the raster

arcpy.ddd.Reclassify("stream_raster", "Value", "1 1;2 2;3 3;4 4;5 5;6 6;8 7;NODATA 0", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\stream_reclass", "NODATA")

# Ready for Cost Surface

# Final raster name 'stream_reclass'

### Elevation

In [ ]:
# Calculate Slope

arcpy.ddd.Slope("DigitalElevationModel_Clip", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Slope_DEM", "DEGREE", 1, "PLANAR", "METER")

# Reclassify

arcpy.ddd.Reclassify("Slope_DEM", "VALUE", "0 1.720000 1;1.720000 3.430000 1;3.430000 5.710000 2;5.710000 8.530000 3;8.530000 11.300000 4;11.300000 14.040000 5;14.040000 16.700000 5;16.700000 21.800000 6;21.800000 30.960000 6;30.960000 45 7;45 90 7;NODATA 0", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\DEM_reclass", "NODATA")

# Ready for Cost Surface

# Final raster name 'DEM_reclass'

### Land Cover

In [ ]:
# Reclassify 

arcpy.ddd.Reclassify(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\NLCD2019LandCover_Clip", "NLCD_Land", "'Open Water' 1;'Developed, Open Space' 2;'Developed, Low Intensity' 2;'Developed, Medium Intensity' 2;'Developed, High Intensity' 2;'Barren Land' 3;'Deciduous Forest' 4;'Evergreen Forest' 4;'Mixed Forest' 4;Shrub/Scrub 5;Herbaceous 5;Hay/Pasture 6;'Cultivated Crops' 7;'Woody Wetlands' 14;'Emergent Herbaceous Wetlands' 7;NODATA 0", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\landcover_reclass", "NODATA")

# Ready for Cost Surface

# Final raster name 'landcover_reclass'

## Cost Surface Analysis

In [ ]:
# Weighted Overlay to try out different weights

## 33(34)% weight for each raster

out_raster = arcpy.sa.WeightedOverlay(r"('C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\landcover_reclass' 33 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 14 1; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\DEM_reclass' 33 'Value' (0 1; 1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\stream_reclass' 34 'Value' (0 1; 1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; NODATA NODATA));1 9 1"); out_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Weighted_1")

## 10% Streams, 30% Elevation, 60% Land Cover

out_raster = arcpy.sa.WeightedOverlay(r"('C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\landcover_reclass' 60 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 14 1; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\DEM_reclass' 30 'Value' (0 1; 1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\stream_reclass' 10 'Value' (0 1; 1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; NODATA NODATA));1 9 1"); out_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Weighted_2")

## 10% Streams, 60% Elevation, 30% Land Cover

out_raster = arcpy.sa.WeightedOverlay(r"('C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\landcover_reclass' 30 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 14 1; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\DEM_reclass' 60 'Value' (0 1; 1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\stream_reclass' 10 'Value' (0 1; 1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; NODATA NODATA));1 9 1"); out_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Weighted_3")

## 60% Streams, 10% Elevation, 30% Land Cover

out_raster = arcpy.sa.WeightedOverlay(r"('C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\landcover_reclass' 30 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 14 1; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\DEM_reclass' 10 'Value' (0 1; 1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; NODATA NODATA); 'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\stream_reclass' 60 'Value' (0 1; 1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; NODATA NODATA));1 9 1"); out_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\Weighted_4")

## Cost Path

In [ ]:
# Path 1 - 33(34)% weight for each raster

# Cost distance to North Picnic

out_distance_raster = arcpy.sa.CostDistance("NorthPicnicXY_XYTableToPoint1", "Weighted_1", None, r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostBL_North", None, None, None, None, ''); out_distance_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostDis_North")

# Back link to Dory's Farm

out_backlink_raster = arcpy.sa.CostBackLink("DoryFarmXY_XYTableToPoint1", "Weighted_1", None, r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostDis_Dory", None, None, None, None, ''); out_backlink_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostBL_Dory")

# Final Cost Path

out_raster = arcpy.sa.CostPath("DoryFarmXY_XYTableToPoint1", "CostDis_Dory", "CostBL_North", "BEST_SINGLE", "OBJECTID", "INPUT_RANGE"); out_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostPath_Dory")

In [ ]:
# Path 2 - 10% Streams, 30% Elevation, 60% Land Cover

# Cost distance to North Picnic

out_distance_raster = arcpy.sa.CostDistance("NorthPicnicXY_XYTableToPoint1", "Weighted_2", None, r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostBL_North2", None, None, None, None, ''); out_distance_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostDis_North2")

# Back link to Dory's Farm

out_backlink_raster = arcpy.sa.CostBackLink("DoryFarmXY_XYTableToPoint1", "Weighted_2", None, r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostDis_Dory2", None, None, None, None, ''); out_backlink_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostBL_Dory2")

# Final Cost Path

out_raster = arcpy.sa.CostPath("DoryFarmXY_XYTableToPoint1", "CostDis_Dory2", "CostBL_North2", "BEST_SINGLE", "OBJECTID", "INPUT_RANGE"); out_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostPath_Dory2")

In [ ]:
# Path 3 - 10% Streams, 60% Elevation, 30% Land Cover

# Cost distance to North Picnic

out_distance_raster = arcpy.sa.CostDistance("NorthPicnicXY_XYTableToPoint1", "Weighted_3", None, r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostBL_North3", None, None, None, None, ''); out_distance_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostDis_North3")

# Back link to Dory's Farm

out_backlink_raster = arcpy.sa.CostBackLink("DoryFarmXY_XYTableToPoint1", "Weighted_3", None, r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostDis_Dory3", None, None, None, None, ''); out_backlink_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostBL_Dory3")

# Final Cost Path

out_raster = arcpy.sa.CostPath("DoryFarmXY_XYTableToPoint1", "CostDis_Dory3", "CostBL_North3", "BEST_SINGLE", "OBJECTID", "INPUT_RANGE"); out_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostPath_Dory3")

In [ ]:
# Path 4 - 60% Streams, 10% Elevation, 30% Land Cover

#Cost distance to North Picnic

out_distance_raster = arcpy.sa.CostDistance("NorthPicnicXY_XYTableToPoint1", "Weighted_4", None, r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostBL_North4", None, None, None, None, ''); out_distance_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostDis_North4")

# Back link to Dory's Farm

out_backlink_raster = arcpy.sa.CostBackLink("DoryFarmXY_XYTableToPoint1", "Weighted_4", None, r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostDis_Dory4", None, None, None, None, ''); out_backlink_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostBL_Dory4")

# Final Cost Path

out_raster = arcpy.sa.CostPath("DoryFarmXY_XYTableToPoint1", "CostDis_Dory4", "CostBL_North4", "BEST_SINGLE", "OBJECTID", "INPUT_RANGE"); out_raster.save(r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\Dory\GIS5571_Lab2Part2\GIS5571_Lab2Part2.gdb\CostPath_Dory4")